# Load packages and import modules

In [ ]:
# -*- coding: utf-8 -*-
import sys
import os

# Manually specify the path to the src folder
sys.path.append(os.path.abspath('../'))

2025-05-02 23:17:39.127194: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-02 23:17:39.127230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-02 23:17:39.128308: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-02 23:17:39.134383: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-02 23:17:39.841606: W tensorflow/compiler/tf2


device: 0, name: NVIDIA GeForce RTX 3050 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


2025-05-02 23:17:40.561469: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-02 23:17:40.598266: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-02 23:17:40.601108: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# Load datasets 

In [ ]:
from core.loader import Loader

benign_dataset_filenames = [
    '../parkets/benign/benign_2312_anonymized_HTML.parquet', 
    '../parkets/benign/umbrella_benign_FINISHED_HTML.parquet'
        
]
malicious_dataset_filenames = [
    '../parkets/malware_2406_strict_HTML.parquet'
]

# CONFIGURATION

benign_label = "benign"
malicious_label = "malware"

class_map = {benign_label: 0, malicious_label: 1}
# print labels from malicious datasets

loader = Loader(benign_dataset_filenames, malicious_dataset_filenames, benign_label=benign_label, malicious_label=malicious_label, subsample=0.08)
df = loader.load()

# Split data into stages

In [ ]:
from core.loader import Segmenter

# Define the aggregates that needs to be created

aggregates = [
    ["lex_"],
    ["lex_", "dns_", "ip_", "geo_"],
    ["lex_", "dns_", "ip_", "tls_", "geo_", "rdap_"],
]

segmenter = Segmenter(df)
segmenter.create_base_subsets() # create base subsets
segmenter.create_aggregated_subsets(aggregates)
subset_dfs = segmenter.get_aggregated_subsets()

/home/poli/Desktop/git/deep_domain_detection/src/core/loader.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["label"] = self.df["label"].copy()
/home/poli/Desktop/git/deep_domain_detection/src/core/loader.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["label"] = self.df["label"].copy()
/home/poli/Desktop/git/deep_domain_detection/src/core/loader.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

# SVM training

In [ ]:
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from models.model_wrapper import ModelWrapper
from sklearn.metrics import classification_report
import numpy as np
from sklearn.svm import SVC

ARCH_NAME = "svm"
VERSION = "v1.1"
LR = 0.0023



# Initialize ModelWrapper 
wrapper = ModelWrapper()

def svm_convert(df):
    df = df.fillna(0)
    df = df.replace({True: 1, False: 0})
    return np.array(df)

i=1
for prefix, subset_df in subset_dfs.items():
    
    print(f"\n🚀 Training SVM on '{prefix}' features…")
    
    labels = subset_df['label'].apply(lambda x: class_map[x]) # y vector
    features = subset_df.drop('label', axis=1).copy() # X matrix
    
    labels = svm_convert(labels)
    features = loader.scale(features, stage=i, model=ARCH_NAME)
    features = svm_convert(features)

    x_train, x_test, y_train, y_test = train_test_split(
        features,
        labels,
        test_size=0.2,
        random_state=42,
        shuffle=True, 
        stratify=labels
    )

    # Following parameters were found by gradient grid search
    params = {
        "C": 59,
        "gamma": 0.1,
        "kernel": "rbf",
        "random_state": 42,
        "class_weight": "balanced"
    }
    
    model = SVC(**params, verbose=True)
    model.fit(x_train, y_train)
    
    
    ### stage evaluation ###
    try:
        # print the model's parameters
        print("Model parameters:")
        for param, value in model.get_params().items():
            print(f"{param}: {value}")
            
        y_pred = model.predict(x_test)
        y_pred_train = model.predict(x_train)
    
        print("Classification report on test set:")
        print(classification_report(y_test, y_pred, target_names=class_map.keys()))
        print("Classification report on train set:")
        print(classification_report(y_train, y_pred_train, target_names=class_map.keys()))
    except Exception as e:
        print(f"Error during model evaluation: {e}")


    wrapper.save(model, arch_name=ARCH_NAME, label=malicious_label, prefix=f"{prefix}", version=VERSION)
    i+=1



Training svm on subset:  lex_agg
[LibSVM]............................................*....................*
optimization finished, #iter = 64289
obj = -31771.082690, rho = -0.399189
nSV = 11546, nBSV = 379
Total nSV = 11546
Model parameters:
C: 59
break_ties: False
cache_size: 200
class_weight: balanced
coef0: 0.0
decision_function_shape: ovr
degree: 3
gamma: 0.1
kernel: rbf
max_iter: -1
probability: False
random_state: 42
shrinking: True
tol: 0.001
verbose: True
Classification report on test set:
              precision    recall  f1-score   support

      benign       0.94      0.96      0.95      6538
    phishing       0.79      0.68      0.73      1308

    accuracy                           0.92      7846
   macro avg       0.86      0.82      0.84      7846
weighted avg       0.91      0.92      0.91      7846

Classification report on train set:
              precision    recall  f1-score   support

      benign       1.00      1.00      1.00     26151
    phishing       0.99  